In [1]:
import random

import matplotlib.pyplot as plt
import numpy as np

from saxs.data_generation.processing import Processing
from saxs.data_generation.data_visualization import load_data, plot_saxs, plot_saxs_featuremap
from saxs.data_generation.generation import Generator
from saxs.data_generation import DEFAULT_CONFIG_PATH

import json

from saxs.gaussian_processing.processing_outils import read_data

with open(DEFAULT_CONFIG_PATH) as config:
    config_data = json.load(config)

/Users/isaigordeev/Desktop/2023/saxs/saxs


In [ ]:
q, d1 = load_data(phase=config_data['phase'],
                                cubic_mesophase=config_data['cubic_mesophase'],
                                )
q = q[:498]
q_0, I_0, dI = read_data('/Users/isaigordeev/Desktop/2023/saxs/res/075775_treated_xye.csv')
I_0 = I_0[:498]
q_0 = q_0[:498]
mean = np.mean(I_0)
var = np.std(I_0)
I_0 -= mean
I_0 /= (var ** 0.5)
I_0 /= np.max(I_0)

In [ ]:
for n in random.sample(range(len(d1)), 10):
    # plot_saxs(q , d1[n])
    plt.plot(q, d1[n]/np.max(d1[n]))
    plt.plot(q_0, I_0/np.max(I_0), 'red')

plt.show()

In [ ]:
image = np.uint8(d1[0]*255)
plt.imshow(np.repeat(np.expand_dims(np.outer(image, image), -1), 3, axis=-1))

In [ ]:
image = np.uint8(I_0/np.max(I_0)*255)
plt.imshow(np.repeat(np.expand_dims(np.outer(image, image), -1), 3, axis=-1))

In [ ]:
for n in random.sample(range(len(d1)), 10):
    # plot_saxs(q , d1[n])
    plt.plot(q, d1[n]/np.max(d1[n]))
    plt.plot(q_0, I_0/np.max(I_0), 'red')

plt.show()

In [ ]:
image = np.uint8(d1[0]*255)
plt.imshow(np.repeat(np.expand_dims(np.outer(image, image), -1), 3, axis=-1))

In [ ]:
len(d1[0][68:])

In [2]:


from saxs.saxs_model.model import SAXSViT10

from torchinfo import summary #optional

from saxs.saxs_model.model_settings import IMAGE_DIM, COLOR_CHANNELS, PATCH_SIZE, ATTENTION_BLOCKS, EMBEDDING_DIM

mod = SAXSViT10(IMAGE_DIM,
                COLOR_CHANNELS,
                PATCH_SIZE,
                3,
                24,
                3072, 12, 0.1, 0, 0.1, 3)

print(mod.patch_embedding)



summary(mod,
        input_size=(1, 3, 498, 498),  # (batch_size, color_channels, height, width)
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"]
        )


24
PatchEmbedding(
  (patcher): Conv2d(3, 24, kernel_size=(166, 166), stride=(166, 166))
  (flatten): Flatten(start_dim=2, end_dim=3)
)


Layer (type (var_name))                  Input Shape          Output Shape         Param #              Trainable
SAXSViT10 (SAXSViT10)                    [1, 3, 498, 498]     [1, 3]               264                  True
├─PatchEmbedding (patch_embedding)       [1, 3, 498, 498]     [1, 9, 24]           --                   True
│    └─Conv2d (patcher)                  [1, 3, 498, 498]     [1, 24, 3, 3]        1,984,056            True
│    └─Flatten (flatten)                 [1, 24, 3, 3]        [1, 24, 9]           --                   --
├─Dropout (embedding_dropout)            [1, 10, 24]          [1, 10, 24]          --                   --
├─Sequential (transformer_encoder)       [1, 10, 24]          [1, 10, 24]          --                   True
│    └─TransformerEncoderLayer (0)       [1, 10, 24]          [1, 10, 24]          153,048              True
│    └─TransformerEncoderLayer (1)       [1, 10, 24]          [1, 10, 24]          153,048              True
│    └─Transformer

In [1]:

import os
from saxs import PACKAGE_PATH, DEFAULT_PHASES_PATH
from saxs.saxs_model.model import SAXSViT10


import matplotlib.pyplot as plt
import torch
from torchvision.transforms import transforms

import saxs.saxs_model.saxs_dataset as data_setup
from saxs import PACKAGE_PATH
import json
import saxs.saxs_model.phase_prediction as phase_prediction
from saxs.saxs_model import engine
from saxs.saxs_model.model import SAXSViT
from saxs.saxs_model.model_settings import DEVICE

from torchinfo import summary #optional

from saxs.saxs_model.model_settings import IMAGE_DIM, COLOR_CHANNELS, PATCH_SIZE, ATTENTION_BLOCKS, EMBEDDING_DIM

with open(DEFAULT_PHASES_PATH, 'r') as file:  # NOTE make it better with string formatting
    phases = json.load(file)

classes = list(phases.keys())
class_to_idx = {cls_name: i for i, cls_name in enumerate(classes)}


mod = SAXSViT10(IMAGE_DIM,
                COLOR_CHANNELS,
                166,
                3,
                24,
                3072, 12, 0.1, 0, 0.1, 3)

# train_saxs_batches, test_saxs_batches, saxs_phases = \
#     data_setup.create_data_batches_from_dataset_files(path=os.path.join(PACKAGE_PATH, 'cache'),
#                                                     transforms=None,
#                                                     batch_size=32,
#                                                     num_workers=0
#                                                      )

phase_prediction.prediction_from_csv(mod,
                            'res/075773_treated_xye.csv',
                            classes,
                            )


/Users/isaigordeev/Desktop/2023/saxs/saxs


/Users/isaigordeev/Desktop/2023/SAXS/venv38/lib/python3.8/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/isaigordeev/Desktop/2023/SAXS/venv38/lib/python3.8/site-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
/Users/isaigordeev/Des

24
SAMPLE IS GREAT


NameError: name 'patch_size' is not defined